In [1]:
import pandas as pd
import numpy as np

# 🏦 Bank Customer Dataset Schema

This notebook documents the schema of a customer dataset used for banking churn analysis. It includes feature descriptions, data types, and inferred meanings to guide preprocessing and modeling.

---

## 📋 Dataset Columns

| Column Name           | Data Type | Description (Inferred)                                      |
|-----------------------|-----------|--------------------------------------------------------------|
| `id`, `CustomerId`    | int64     | Unique identifiers for each customer.                        |
| `Surname`             | object    | Customer's last name.                                        |
| `CreditScore`         | int64     | Credit score of the customer.                                |
| `Geography`, `Gender` | object    | Categorical features representing location and gender.       |
| `Age`                 | float64   | Customer's age.                                              |
| `Tenure`              | int64     | Number of years the customer has been with the bank.         |
| `Balance`             | float64   | Account balance.                                             |
| `EstimatedSalary`     | float64   | Estimated annual salary.                                     |
| `NumOfProducts`       | int64     | Number of bank products the customer uses.                   |
| `HasCrCard`           | float64   | Binary flag: 1 if customer has a credit card, else 0.        |
| `IsActiveMember`      | float64   | Binary flag: 1 if customer is active, else 0.                |
| `Exited`              | int64     | Target variable: 1 if customer exited (churned), else 0.     |

---

## 🧭 Notes

- Binary flags (`HasCrCard`, `IsActiveMember`) are stored as `float64`, which may require conversion to `int` or `bool` for modeling.
- Categorical features (`Geography`, `Gender`) may need encoding (e.g., one-hot or label encoding).
- `Exited` is the target variable for churn prediction.

---

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.drop(columns=['id' , 'CustomerId' , 'Surname'],inplace=True)

In [4]:
categorical_features = df.select_dtypes(include='O').columns

In [5]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

# Applying OHE in Geography and Ordinal Encoder on Gender Feature

Female = 0

Male = 1

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...
165029,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [7]:
from sklearn.preprocessing import StandardScaler , OneHotEncoder , OrdinalEncoder

In [8]:
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer

In [9]:
cols = df.columns

In [10]:
X = df.drop(columns='Exited')
y = df['Exited']

In [11]:
preprocessing = ColumnTransformer(
    transformers=[
        ('Gender_Ordinal_Encoder' , OrdinalEncoder() , ['Gender']),
        ('Geography_OHE' , OneHotEncoder(handle_unknown='ignore' , sparse_output=False) , ['Geography'])
    ],remainder=StandardScaler()
)

In [12]:
preprocessed_data = preprocessing.fit_transform(X)

In [13]:
preprocessing

,transformers,"[('Gender_Ordinal_Encoder', ...), ('Geography_OHE', ...)]"
,remainder,StandardScaler()
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'


In [14]:
preprocessing.get_feature_names_out()

array(['Gender_Ordinal_Encoder__Gender',
       'Geography_OHE__Geography_France',
       'Geography_OHE__Geography_Germany',
       'Geography_OHE__Geography_Spain', 'remainder__CreditScore',
       'remainder__Age', 'remainder__Tenure', 'remainder__Balance',
       'remainder__NumOfProducts', 'remainder__HasCrCard',
       'remainder__IsActiveMember', 'remainder__EstimatedSalary'],
      dtype=object)

In [15]:
X_preprocessed = pd.DataFrame(preprocessed_data , columns=preprocessing.get_feature_names_out())

In [16]:
df.shape

(165034, 11)

In [17]:
X_preprocessed

,Gender_Ordinal_Encoder__Gender,Geography_OHE__Geography_France,Geography_OHE__Geography_Germany,Geography_OHE__Geography_Spain,remainder__CreditScore,remainder__Age,remainder__Tenure,remainder__Balance,remainder__NumOfProducts,remainder__HasCrCard,remainder__IsActiveMember,remainder__EstimatedSalary
0,1.0,1.0,0.0,0.0,0.144135,-0.578074,-0.719973,-0.883163,0.814298,0.571263,-0.99555,1.369486
1,1.0,1.0,0.0,0.0,-0.367706,-0.578074,-1.432694,-0.883163,0.814298,0.571263,1.00447,-1.254085
2,1.0,1.0,0.0,0.0,0.268974,0.211354,1.774548,-0.883163,0.814298,0.571263,-0.99555,1.437422
3,1.0,1.0,0.0,0.0,-0.941966,-0.465299,-1.076334,1.486918,-1.013348,0.571263,1.00447,-0.557018
4,1.0,0.0,0.0,1.0,0.743362,-0.578074,-0.007253,-0.883163,0.814298,0.571263,1.00447,-1.938770
...,...,...,...,...,...,...,...,...,...,...,...,...
165029,0.0,0.0,0.0,1.0,0.131651,-0.578074,-1.076334,-0.883163,-1.013348,0.571263,1.00447,0.382957
165030,1.0,1.0,0.0,0.0,1.692140,-0.352523,-0.719973,-0.883163,-1.013348,-1.750507,-0.99555,0.382951
165031,1.0,1.0,0.0,0.0,-1.141708,-0.803625,-0.007253,-0.883163,-1.013348,0.571263,1.00447,0.295366
165032,0.0,0.0,0.0,1.0,-1.279031,-0.916401,0.705467,1.688303,-1.013348,-1.750507,1.00447,-0.823217


In [18]:
import pickle

with open('preprocessed.pkl' , 'wb') as file:
    pickle.dump(preprocessing,file)

# Training The Model

In [19]:
cols = X_preprocessed.columns

In [20]:
cols

Index(['Gender_Ordinal_Encoder__Gender', 'Geography_OHE__Geography_France',
       'Geography_OHE__Geography_Germany', 'Geography_OHE__Geography_Spain',
       'remainder__CreditScore', 'remainder__Age', 'remainder__Tenure',
       'remainder__Balance', 'remainder__NumOfProducts',
       'remainder__HasCrCard', 'remainder__IsActiveMember',
       'remainder__EstimatedSalary'],
      dtype='object')

In [21]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X_preprocessed , y , test_size=0.2 , random_state=42)

# Applying RandomForest on this dataset

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
RFC = RandomForestClassifier(n_estimators=100 , oob_score=True)
RFC.fit(X_train , y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,True


In [24]:
y_pred = RFC.predict(X_test)

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.8573938861453632

# Applying XGBoost ALgo

In [26]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train , y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [27]:
y_pred = xgb.predict(X_test)

In [28]:
accuracy_score(y_test , y_pred)

0.8653012997243009

# Applying ANN

In [29]:
import tensorflow
from tensorflow import keras 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense , Dropout , BatchNormalization
from tensorflow.keras.regularizers import l2

import keras_tuner as kt

In [30]:
X_train.shape

(132027, 12)

In [31]:
def build_model(hp):
    model = Sequential()
    
    # num of layers
    layers = hp.Int('hiddenLayer' , min_value = 1 , max_value = 10)

    for i in range(layers):
        unit = hp.Int(f'Neurons_{i}' , min_value = 8 , max_value = 256)
        activation = hp.Choice(f'Activation_{i}' , values = ['relu' , 'leaky_relu'])
        regularizer = hp.Choice(f'Regularizer_{i}' , values = [0.001 , 0.0001 , 0.0003])
        wtInit = hp.Choice(f"wt_Init_{i}" , values = ['he_normal' , 'he_uniform'])
        dropout = hp.Choice(f'DroupOut_{i}' , values = [0.3 , 0.4 , 0.5])

        if i == 0 :
            model.add(
                Dense(
                    unit , 
                    activation = activation , 
                    input_dim = X_train.shape[1] , 
                    kernel_regularizer = l2(regularizer) ,
                    kernel_initializer = wtInit
                )
            )
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
        
        else:
            model.add(
                Dense(
                    unit , 
                    activation = activation , 
                    kernel_regularizer = l2(regularizer) ,
                    kernel_initializer = wtInit
                )
            )
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
        
    # Output Layer
    model.add(Dense(1 , activation = 'sigmoid')) 
    
    lr = hp.Choice('optimizer_Lr' , values = [0.0001 , 0.0003])
    optimizers = keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss = 'binary_crossentropy' , optimizer=optimizers , metrics=['accuracy'])

    return model

In [32]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 10,
    directory = 'Model_Tuner',
    project_name = 'Churn Prediction'
)

Reloading Tuner from Model_Tuner\Churn Prediction\tuner0.json


In [33]:
tuner.search(X_train , y_train , epochs = 10 , validation_data = (X_test , y_test))

Trial 10 Complete [00h 01m 59s]
val_accuracy: 0.8402156829833984

Best val_accuracy So Far: 0.864725649356842
Total elapsed time: 01h 31m 44s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'hiddenLayer': 4,
 'Neurons_0': 86,
 'Activation_0': 'relu',
 'Regularizer_0': 0.001,
 'wt_Init_0': 'he_uniform',
 'DroupOut_0': 0.4,
 'optimizer_Lr': 0.0003,
 'Neurons_1': 173,
 'Activation_1': 'leaky_relu',
 'Regularizer_1': 0.001,
 'wt_Init_1': 'he_uniform',
 'DroupOut_1': 0.3,
 'Neurons_2': 238,
 'Activation_2': 'leaky_relu',
 'Regularizer_2': 0.0003,
 'wt_Init_2': 'he_uniform',
 'DroupOut_2': 0.4,
 'Neurons_3': 121,
 'Activation_3': 'relu',
 'Regularizer_3': 0.0001,
 'wt_Init_3': 'he_normal',
 'DroupOut_3': 0.4,
 'Neurons_4': 207,
 'Activation_4': 'relu',
 'Regularizer_4': 0.0001,
 'wt_Init_4': 'he_normal',
 'DroupOut_4': 0.3,
 'Neurons_5': 15,
 'Activation_5': 'leaky_relu',
 'Regularizer_5': 0.001,
 'wt_Init_5': 'he_uniform',
 'DroupOut_5': 0.4,
 'Neurons_6': 111,
 'Activation_6': 'leaky_relu',
 'Regularizer_6': 0.0001,
 'wt_Init_6': 'he_normal',
 'DroupOut_6': 0.3,
 'Neurons_7': 253,
 'Activation_7': 'relu',
 'Regularizer_7': 0.0003,
 'wt_Init_7': 'he_uniform',
 'DroupOut_7': 0

In [35]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\LeoML\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\LeoML\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [36]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 86)             │         1,118 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 86)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 86)             │           344 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 173)            │        15,051 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 173)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 173)            │           692 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 238)            │        41,412 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 238)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 238)            │           952 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 121)            │        28,919 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 121)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 121)            │           484 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           122 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,094 (348.02 KB)

 Trainable params: 87,858 (343.20 KB)

 Non-trainable params: 1,236 (4.83 KB)

In [37]:
from tensorflow.keras.callbacks import EarlyStopping

callback = EarlyStopping(
    monitor = 'val_accuracy',
    min_delta = 0.001,
    patience = 20 ,
    mode = 'auto',
    baseline = None
)

In [38]:
model.fit(
    X_train ,
    y_train ,
    epochs=100 , 
    initial_epoch=10 ,
    callbacks = callback ,
    validation_data=(X_test ,y_test)
)

Epoch 11/100
4126/4126 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8571 - loss: 0.3562 - val_accuracy: 0.8643 - val_loss: 0.3419
Epoch 12/100
4126/4126 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8581 - loss: 0.3550 - val_accuracy: 0.8632 - val_loss: 0.3409
Epoch 13/100
4126/4126 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8576 - loss: 0.3540 - val_accuracy: 0.8647 - val_loss: 0.3386
Epoch 14/100
4126/4126 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8576 - loss: 0.3533 - val_accuracy: 0.8645 - val_loss: 0.3384
Epoch 15/100
4126/4126 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8573 - loss: 0.3521 - val_accuracy: 0.8622 - val_loss: 0.3394
Epoch 16/100
4126/4126 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8584 - loss: 0.3521 - val_accuracy: 0.8629 - val_loss: 0.3384
Epoch 17/100
4126/4126 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8586 - loss: 0.3510 - val_accuracy: 0.8614 - val_loss: 0.3406
Epoch 18/100
4126/4126 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8586 

In [39]:
model.save('model.keras')